In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import os
import cProfile

[09/20/23 19:21:32] INFO     Custom team name replacements loaded from                                ]8;id=117295;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=811314;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#82\82]8;;\
                             /home/morten/soccerdata/config/teamname_replacements.json.                            

                    INFO     Custom league dict loaded from                                          ]8;id=758193;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=681712;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#160\160]8;;\
                             /home/morten/soccerdata/config/league_dict.json.                                      

In [3]:
from game_timeline import GameTimeline
import player
import elo

In [4]:
ws = sd.WhoScored(
    #  "GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"
    leagues=["GER-Bundesliga"],
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=False,
)

[09/20/23 19:21:34] INFO     Saving cached data to /home/morten/Develop/Open-Data/soccerdata          ]8;id=694258;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=956593;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_common.py#89\89]8;;\

[09/20/23 19:21:35] INFO     patching driver executable                                              ]8;id=23744;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/undetected_chromedriver/patcher.py\patcher.py]8;;\:]8;id=595948;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/undetected_chromedriver/patcher.py#298\298]8;;\
                             /home/morten/.local/share/undetected_chromedriver/undetected_chromedriv               
                             er                                                                                    

[09/20/23 19:21:36] ERROR                                                                            ]8;id=795802;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=517629;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_common.py#389\389]8;;\
                                             The ChromeDriver was unable to initiate/spawn a new                   
                                             WebBrowser. You will not be able to scrape new data.                  
                                             Message: unknown error: cannot connect to chrome at                   
                             127.0.0.1:59271                                                                       
                             from session not created: This version of ChromeDriver only supports                  
                             Chrome version 114                                                                    
                             Current browser version is 117.0.5938.88                                              
                             Stacktrace:                                                                           
                             #0 0x55e2b68dd4e3 <unknown>                                                           
                             #1 0x55e2b660cc76 <unknown>                                                           
                             #2 0x55e2b663a04a <unknown>                                                           
                             #3 0x55e2b6631884 <unknown>                                                           
                             #4 0x55e2b6670ccc <unknown>                                                           
                             #5 0x55e2b667047f <unknown>                                                           
                             #6 0x55e2b6667de3 <unknown>                                                           
                             #7 0x55e2b663d2dd <unknown>                                                           
                             #8 0x55e2b663e34e <unknown>                                                           
                             #9 0x55e2b689d3e4 <unknown>                                                           
                             #10 0x55e2b68a13d7 <unknown>                                                          
                             #11 0x55e2b68abb20 <unknown>                                                          
                             #12 0x55e2b68a2023 <unknown>                                                          
                             #13 0x55e2b68701aa <unknown>                                                          
                             #14 0x55e2b68c66b8 <unknown>                                                          
                             #15 0x55e2b68c6847 <unknown>                                                          
                             #16 0x55e2b68d6243 <unknown>                                                          
                             #17 0x7fdffc2f29eb <unknown>                                                          
                                                                                                                   
                                                                                                                   

In [5]:
schedule = ws.read_schedule()
schedule.head()

[09/20/23 19:21:50] INFO     Retrieving game schedule of GER-Bundesliga - 2122 from the cache      ]8;id=248748;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/whoscored.py\whoscored.py]8;;\:]8;id=244392;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/whoscored.py#437\437]8;;\

date  \
league         season game                                                                   
GER-Bundesliga 2122   2021-08-13 Borussia M.Gladbach-Bayern            2021-08-13 19:30:00   
                      2021-08-14 Arminia Bielefeld-Freiburg            2021-08-14 14:30:00   
                      2021-08-14 Augsburg-Hoffenheim                   2021-08-14 14:30:00   
                      2021-08-14 Borussia Dortmund-Eintracht Frankfurt 2021-08-14 17:30:00   
                      2021-08-14 Stuttgart-Greuther Fuerth             2021-08-14 14:30:00   

                                                                                  home_team  \
league         season game                                                                    
GER-Bundesliga 2122   2021-08-13 Borussia M.Gladbach-Bayern             Borussia M.Gladbach   
                      2021-08-14 Arminia Bielefeld-Freiburg               Arminia Bielefeld   
                      2021-08-14 Augsburg-Hoffenheim                               Augsburg   
                      2021-08-14 Borussia Dortmund-Eintracht Frankfurt    Borussia Dortmund   
                      2021-08-14 Stuttgart-Greuther Fuerth                        Stuttgart   

                                                                                  away_team  \
league         season game                                                                    
GER-Bundesliga 2122   2021-08-13 Borussia M.Gladbach-Bayern                          Bayern   
                      2021-08-14 Arminia Bielefeld-Freiburg                        Freiburg   
                      2021-08-14 Augsburg-Hoffenheim                             Hoffenheim   
                      2021-08-14 Borussia Dortmund-Eintracht Frankfurt  Eintracht Frankfurt   
                      2021-08-14 Stuttgart-Greuther Fuerth                  Greuther Fuerth   

                                                                        game_id  \
league         season game                                                        
GER-Bundesliga 2122   2021-08-13 Borussia M.Gladbach-Bayern             1557324   
                      2021-08-14 Arminia Bielefeld-Freiburg             1557326   
                      2021-08-14 Augsburg-Hoffenheim                    1557327   
                      2021-08-14 Borussia Dortmund-Eintracht Frankfurt  1557332   
                      2021-08-14 Stuttgart-Greuther Fuerth              1557329   

                                                                                                                      url  \
league         season game                                                                                                  
GER-Bundesliga 2122   2021-08-13 Borussia M.Gladbach-Bayern             https://www.whoscored.com/Matches/1557324/Live...   
                      2021-08-14 Arminia Bielefeld-Freiburg             https://www.whoscored.com/Matches/1557326/Live...   
                      2021-08-14 Augsburg-Hoffenheim                    https://www.whoscored.com/Matches/1557327/Live...   
                      2021-08-14 Borussia Dortmund-Eintracht Frankfurt  https://www.whoscored.com/Matches/1557332/Live...   
                      2021-08-14 Stuttgart-Greuther Fuerth              https://www.whoscored.com/Matches/1557329/Live...   

                                                                        stage  
league         season game                                                     
GER-Bundesliga 2122   2021-08-13 Borussia M.Gladbach-Bayern               NaN  
                      2021-08-14 Arminia Bielefeld-Freiburg               NaN  
                      2021-08-14 Augsburg-Hoffenheim                      NaN  
                      2021-08-14 Borussia Dortmund-Eintracht Frankfurt    NaN  
                      2021-08-14 Stuttgart-Greuther Fuerth                NaN

In [6]:
def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp).replace(tzinfo=timezone.utc)

In [8]:
from db_handler import DB_handler

In [9]:
from tqdm import tqdm
def exec(schedule):
    schedule = schedule.sort_values("date")
    for game_id, date in (pbar := tqdm(zip(schedule.game_id, schedule.date), total=len(schedule.game_id))):
        pbar.set_description(f"processing game_id:{game_id}")
        date = to_datetime(date)
        dbh = DB_handler("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/GDE.db")
        game_timeline = GameTimeline(ws, game_id, date, dbh)
        for player_id in game_timeline.general_info_dict:
            player_name = game_timeline.general_info_dict[player_id]["player_name"]
            team_id = game_timeline.general_info_dict[player_id]["team_id"]
            team_name = game_timeline.general_info_dict[player_id]["team_name"]
            proto_player = player.Player(int(player_id), player_name, team_id, team_name, game_timeline.game_date)
            proto_player.add_game(game_timeline) 


In [11]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
# cProfile.run("exec(schedule)", 'restats')
exec(schedule)

processing game_id:1557324:   0%|          | 0/306 [00:00<?, ?it/s]

processing game_id:1557324:   0%|          | 0/306 [00:00<?, ?it/s]

2021/2022
1
[('17-12-88',)]
Borussia M.Gladbach 1 2021/2022
2021/2022
1
[('27-08-92',)]
Borussia M.Gladbach 18 2021/2022
2021/2022
1
[('19-01-94',)]
Borussia M.Gladbach 28 2021/2022
2021/2022
1
[('30-09-96',)]
Borussia M.Gladbach 30 2021/2022
2021/2022
1
[('31-12-02',)]
Borussia M.Gladbach 29 2021/2022
2021/2022
1
[('16-03-97',)]
Borussia M.Gladbach 32 2021/2022
2021/2022
1
[('19-02-91',)]
Borussia M.Gladbach 6 2021/2022
2021/2022
1
[('12-02-91',)]
Borussia M.Gladbach 7 2021/2022
2021/2022
1
[('26-08-88',)]
Borussia M.Gladbach 13 2021/2022
2021/2022
1
[('16-04-99',)]
Borussia M.Gladbach 11 2021/2022
2021/2022
1
[('10-03-93',)]
Borussia M.Gladbach 14 2021/2022
2021/2022
1
[('27-03-86',)]
Bayern 1 2021/2022
2021/2022
1
[('02-04-00',)]
Bayern 44 2021/2022
2021/2022
1
[('27-10-98',)]
Bayern 2 2021/2022
2021/2022
1
[('03-09-95',)]
Bayern 4 2021/2022
2021/2022
1
[('02-11-00',)]
Bayern 19 2021/2022
2021/2022
1
[('08-02-95',)]
Bayern 6 2021/2022
2021/2022
1
[('06-02-95',)]
Bayern 8 2021/2022
2

TypeError: Player.__init__() missing 2 required positional arguments: 'kit_number' and 'game_date'

In [ ]:
# load all players
ids, elos, names, unc, elo_be = [], [], [], [], []
for f in os.listdir("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/proto_db"):
    p = player.PlayerProto().parse(open(f"/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/proto_db/{f}", "rb").read())
    appended = False
    try:
        if p.game[-1].uncertainty < 50:
            elos.append(p.game[-1].elo)
            unc.append(p.game[-1].uncertainty)
            elo_be.append(p.game[-1].elo - p.game[-1].uncertainty)
            appended = True
    except:
        pass
    if appended:
        ids.append(f)
        names.append(p.player_name)

elo_frame = pd.DataFrame({"id": ids, "elo":elos, "uncertainty": unc, "name":names, "elo_be": elo_be})

In [ ]:
elo_frame.sort_values("elo", ascending=False).tail(50)

,id,elo,uncertainty,name,elo_be
1441,300705.pdata,209.962646,10.888453,Lucas Tousart,199.074193
403,324951.pdata,209.923553,8.905194,Dirk Carlson,201.018359
284,391446.pdata,209.898300,11.041724,Maurice Trapp,198.856576
116,419942.pdata,209.751709,15.547083,Thomas Keller,194.204626
471,106049.pdata,209.265839,10.304904,Ewerton,198.960935
38,115996.pdata,208.881226,10.556483,Omar Mascarell,198.324742
922,330537.pdata,208.811386,7.465506,Nicklas Shipnoski,201.345881
1537,108770.pdata,208.593216,6.149417,Simon Falette,202.443799
1085,101550.pdata,208.543350,10.414198,Kevin Wimmer,198.129152
483,109476.pdata,208.537445,10.475444,Andreas Wiegel,198.062001


In [ ]:
p = player.PlayerProto().parse(open(f"/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/proto_db/13754.pdata", "rb").read())

In [ ]:
val = []
k = []
date = []
gd = []
opp_elo = []
game_ids = []

for g in p.game:
    val.append(g.elo)
    k.append(g.uncertainty)
    date.append(g.game_date)
    gd.append(g.goaldifference)
    opp_elo.append(g.oppaverageelo)
    game_ids.append(g.game_id)

elo_diff = []
elo_diff.append(val[0])
for x in range(1, len(val)):
   elo_diff.append(val[x] - val[x-1]) 

plot_df = pd.DataFrame({"Game_id": game_ids, "Elo": val, "elo_diff": elo_diff, "k": k, "date":date, "goal_diff": gd, "opp_elo": opp_elo})
plot_df

,Game_id,Elo,elo_diff,k,date,goal_diff,opp_elo
0,1294545.0,261.000000,261.000000,22.000000,2018-08-24 19:30:00+00:00,2,250.000000
1,1294558.0,265.987183,4.987183,11.200000,2018-09-01 17:30:00+00:00,3,241.913528
2,1294563.0,271.413055,5.425873,12.498718,2018-09-15 14:30:00+00:00,2,242.962006
3,1294572.0,276.364075,4.951019,12.350473,2018-09-22 17:30:00+00:00,2,236.511658
4,1294581.0,275.371307,-0.992767,12.652163,2018-09-25 19:30:00+00:00,0,248.875031
...,...,...,...,...,...,...,...
115,1557533.0,386.068542,2.103271,12.491876,2022-04-09 14:30:00+00:00,1,245.232910
116,1557535.0,387.924805,1.856262,12.755162,2022-04-17 14:30:00+00:00,1,232.320618
117,1557569.0,391.499756,3.574951,12.464788,2022-04-23 17:30:00+00:00,2,308.799896
118,1557605.0,386.845856,-4.653900,12.848088,2022-05-08 16:30:00+00:00,0,232.209381


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Waterfall(orientation = "v", y=plot_df["elo_diff"], x=plot_df["date"], customdata=np.stack((plot_df["Game_id"], plot_df["k"], plot_df["goal_diff"], plot_df["opp_elo"]), axis=-1),
                           hovertemplate ='<b>Elo diff:%{y}<br>'+
                                        '<b>Game Id:%{customdata[0]}<br>'+
                                        '<b>K: %{customdata[1]:.2f}<br>'+
                                        '<b>Goal diff:%{customdata[2]}<br>' + 
                                        '<b>Opp elo:%{customdata[3]:.2f}<br>'))
